# Notes K-matrix

- $K_{ij} \iff$ energy flow from state $j$ to state $i$

## parallel (DAS)
Energy leaves system directly

Full matrix $\iff$ Reduced maxtrix: 

- $K_{ij} = 0 ~ \forall i \neq j$

## sequntial (EAS)
Energy flows from one compartment $i$ to the next $i+1$ until it leaves the system ($K_{nn}$)

Assume $\mathbf{K}$ is $n \times n$ and sequence is in order

Full matrix:

- $K_{i+1i} = -K_{ii} ~ \forall i < n \land K_{nn} \neq 0$

Reduced matrix:

- $K^{Reduced}_{i+1i} = K^{Full}_{i+1i} \land K_{ii} = 0 ~ \forall i < n \land K_{nn} \neq 0$

## General
Full matrix:

- $K_{ii} \iff -$ sum flows out of the compartment $i$ (to other compartments or out of the system)
- $K_{ij} ~ \forall i \neq j ~ \iff$ flow from compartment $j$ to compartment $i$

Reduced matrix:

- $\left(\sum _{k\neq i}^n K_{ki}\right)- K^{Full}_{ii} \iff$ flow of the compartment $i$ out of the system
- $K^{Reduced}_{ij} = K^{Full}_{ij} ~ \forall i \neq j ~\iff$ flow from compartment $j$ to compartment $i$

# cyano-PS1 case study

As in the lecture note we first try a global analysis

## Global analysis

Assumpotion the process can ne described by a simple sequential model.

### K-matrix sequential

The full K-matrix for a 4 compartment sequential scheme has the form of $\mathbf{K}^{Full}$

$$
\mathbf{K}^{Full} = \begin{bmatrix} 
    -k_{21} &         &        &           \\ 
    k_{21}  & -k_{32} &        &           \\
            & k_{32}  & -k_{43} &          \\
            &         & k_{43} & k_{4 Out} \\
    \end{bmatrix}
$$

Since both glotaran and pyglotaran use a reduced form of the K-matrix, it need to be transformed

$$
\mathbf{K}^{Reduced} = \begin{bmatrix} 
           &         &        &           \\ 
    k_{21} &         &        &           \\
           &  k_{32} &        &           \\
           &         & k_{43} & k_{4 Out} \\
    \end{bmatrix}
$$


### Notebook helper function for pretty output

In [ ]:
from IPython.display import Markdown
from IPython.display import display


def print_md(markdown_printable):
    """Pretty render Markdown."""
    display(Markdown(str(markdown_printable)))


def print_yaml_file(file_path):
    """Pretty render yaml file."""
    with open(file_path) as f:
        print_md(f"```yaml\n{f.read()}\n```")

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
from glotaran.project.scheme import Scheme

### Read data

In [ ]:
dataset = load_dataset("streakdata.ascii")
dataset

### Load model and parameters

In [ ]:
global_model = load_model("models/global-model.yml")
global_parameters = load_parameters("models/global-parameters.yml")
print_md(global_model.validate(parameters=global_parameters))

In [ ]:
print_md(global_model)

### Create scheme and optimize it

In [ ]:
global_scheme = Scheme(
    global_model,
    global_parameters,
    {"dataset1": dataset},
    non_negative_least_squares=True,
)
global_result = optimize(global_scheme)

In [ ]:
global_result.data["dataset1"]

### Result plots

In [ ]:
fig = plot_overview(global_result.data["dataset1"], linlog=False)

In [ ]:
print_md(global_result.optimized_parameters)

## Target analysis

Theoretical background from ``compartmental_models.pdf``

![model_scheme](target_scheme.jpg)


Concentration vector

$$c(t)= \begin{bmatrix} S(t) & B(t) & R_1(t) & R_2(t) & F(t)\end{bmatrix}^T$$

The concentration function correspond to

| ***Function name***    | ***S(t)***   | ***B(t)***   | ***R_1(t)***   | ***R_2(t)***   | ***F(t)***   |
|------------------------|--------------|--------------|----------------|----------------|--------------|
| **Name in the schema** | Soret        | Bulk         | Red1           | Red2           | Free         |


Differential equation

$$\dfrac{\mathrm{d}}{\mathrm{dt}}c(t) = \mathbf{K}c(t)+j(t)$$

Input function

$$j(t) = i(t) \begin{bmatrix} 1&0&0&0&0\end{bmatrix}^T$$

Transfer maxtrix $\mathbf{K}$

$$
\mathbf{K} = \begin{bmatrix} 
    -(k_{21} + k_{31} + k_{41} + k_{51}) &                          &                   &                  &       \\ 
    k_{21}                               & -(k_T + k_{32} + k_{42}) & k_{23}            & k_{24}           &       \\
    k_{31}                               &  k_{32}                  & -(k_{F} + k_{23}) &                  &       \\
    k_{41}                               &  k_{42}                  &                   & -(k_{F} + k_{24}) &      \\
    k_{51}                               &                          &                   &                  & k_{F} \\
    \end{bmatrix}
$$

- $k_T$: effective rate constant of Chl, T stands for photochemical Trapping of the excitation energy
- $k_F$: natural decay rate of free Chl, F stands for Fluorescence

In [ ]:
# Just a little helper to quickly make a pretty table
from tabulate import tabulate

headers = list(
    map(
        lambda x: f"***{x}***",
        ["Function name", "S(t)", "B(t)", "R_1(t)", "R_2(t)", "F(t)"],
    )
)
print(
    tabulate(
        [["**Name in the schema**", "Soret", "Bulk", "Red1", "Red2", "Free"]],
        headers=headers,
        tablefmt="github",
    )
)

### Reduced K-matrix

$$
\mathbf{K} = \begin{bmatrix} 
           &        &        &        &       \\ 
    k_{21} & k_T    & k_{23} & k_{24} &       \\
    k_{31} & k_{32} & k_{F}  &        &       \\
    k_{41} & k_{42} &        & k_{F}  &       \\
    k_{51} &        &        &        & k_{F} \\
    \end{bmatrix}
$$

### Reduced K-matrix ignoring input to Red1 and Red2

$$
\mathbf{K} = \begin{bmatrix} 
           &        &        &        &       \\ 
    k_{21} & k_T    & k_{23} & k_{24} &       \\
           & k_{32} & k_{F}  &        &       \\
           & k_{42} &        & k_{F}  &       \\
    k_{51} &        &        &        & k_{F} \\
    \end{bmatrix}
$$

With the rate relations:

- $k_{23} = a_1~k_{32}$
- $k_{24} = a_2~k_{42}$

### target-model.yml

In [ ]:
print_yaml_file("models/target-model.yml")

### target-parameters.yml

In [ ]:
print_yaml_file("models/target-parameters.yml")

In [ ]:
target_model = load_model("models/target-model.yml")
target_parameters = load_parameters("models/target-parameters.yml")
print_md(target_model.validate(parameters=target_parameters))

In [ ]:
print_md(target_model)

### Create scheme and optimize it

In [ ]:
target_scheme = Scheme(
    target_model,
    target_parameters,
    {"dataset1": dataset},
    non_negative_least_squares=True,
)
target_result = optimize(target_scheme)

In [ ]:
target_result.data["dataset1"]

### Result plots

In [ ]:
fig = plot_overview(target_result.data["dataset1"], linlog=False)

In [ ]:
print_md(target_result.optimized_parameters)